# Problem with blosc2_append_buffer

In [3]:
import pandas as pd
import tables as tb
# import matplotlib.pyplot as plt
import pycblosc2 as cb2
import numpy as np
from math import log
import time as t

def file_reader(filename, dataset):
    with tb.open_file(filename) as f:
        child = f.root
        for element in dataset.split('/'):
            child = child.__getattr__(element)
        return child[:]
    

KB = 1024
MB = 1024 * KB
GB = 1024 * MB

BLOSC_MAX_FILTERS = 5
BLOSC_BLOCKSIZE = 64 * KB

cparams = cb2.blosc2_create_cparams(compcode=1, clevel=5, use_dict=0, typesize=4,
                                    nthreads=4, blocksize= BLOSC_BLOCKSIZE, schunk=None,
                                    filters=[0, 0, 0, 0, 1],
                                    filters_meta=[0, 0, 0, 0, 0])

dparams = cb2.blosc2_create_dparams(nthreads=1, schunk=None)

data = np.load("../../../outfile.npy")

div_h = 2 ** np.arange(7)
h = len(data)

div_w = 2 ** np.arange(9)
w = len(data[0])

isize = w * h * data.dtype.itemsize
dsize = 0


# Calculo de el orden de la permutacion

TAM_DATA = 4
DATA_SIZE = BLOSC_BLOCKSIZE/TAM_DATA


if  log(DATA_SIZE, 2) % log(TAM_DATA, 2) == 0:
    orden = int(log(DATA_SIZE, TAM_DATA) + 1)
else:
    orden = int(log(DATA_SIZE, 2) + log(TAM_DATA, 2))


for code in [4]:

    cparams.compcode = code   
    
    for dh in div_h:
        for dw in div_w:
            n = []
            ratio = []
            speed = []
            
            start = t.perf_counter()
            
            s_data = np.empty((w, h), dtype=np.float32).flatten()
            
            cparams.filters[BLOSC_MAX_FILTERS - 1] = 0
            cont = 0
            for i in range(h // dh):
                for j in range(w // dw):
                    s_data[cont*dw*dh:(cont+1)*dw*dh] = data[dh*i:dh*(i+1), dw*j:dw*(j+1)].reshape((1,dw*dh))
                    cont += 1
            
            schunk = cb2.blosc2_new_schunk(cparams, dparams)
            nchunks = cb2.blosc2_append_buffer(schunk, isize, s_data)

            end = t.perf_counter()
            
            print('Time: {0:.4f} secs.'.format(end - start))



Time: 0.2184 secs.
Time: 0.1113 secs.
Time: 0.0617 secs.
Time: 0.0404 secs.
Time: 0.0311 secs.
Time: 0.0127 secs.
Time: 0.0088 secs.
Time: 0.0080 secs.
Time: 0.0069 secs.
Time: 0.1189 secs.
Time: 0.0712 secs.
Time: 0.0388 secs.
Time: 0.0218 secs.
Time: 0.0132 secs.
Time: 0.0099 secs.
Time: 0.0066 secs.
Time: 0.0087 secs.
Time: 0.0105 secs.
Time: 0.1460 secs.
Time: 0.0662 secs.
Time: 0.0197 secs.
Time: 0.0143 secs.
Time: 0.0082 secs.
Time: 0.0057 secs.
Time: 0.0047 secs.
Time: 0.0046 secs.
Time: 0.0044 secs.
Time: 0.0321 secs.
Time: 0.0214 secs.
Time: 0.0130 secs.
Time: 0.0086 secs.
Time: 0.0127 secs.
Time: 0.0120 secs.
Time: 0.0054 secs.
Time: 0.0058 secs.
Time: 0.0046 secs.
Time: 0.0188 secs.
Time: 0.0126 secs.
Time: 0.0096 secs.
Time: 0.0076 secs.
Time: 0.0062 secs.
Time: 0.0065 secs.
Time: 0.0057 secs.
Time: 0.0048 secs.
Time: 0.0042 secs.
Time: 0.0123 secs.
Time: 0.0240 secs.
Time: 0.0209 secs.
Time: 0.0177 secs.
Time: 0.0183 secs.
Time: 0.0226 secs.
Time: 0.0210 secs.
Time: 0.0171